In [73]:
import numpy as np
import pandas as pd
import re
import json
import time

from IPython.display import display
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from collections import Counter

# For cralwer
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import xml.etree.ElementTree as et
import requests

import xml.etree.ElementTree as et
import xmltodict

# For visualization
from geopy.geocoders import Nominatim
import folium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24804\873428652.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Censter Listup

In [26]:
def query(post):
    out_ = df_list_1[df_list_1["주소"]==post].reset_index()#
    if len(out_) != 1:
        try:
            result_ = ["multi",out_["지원센터코드"][0]]
        except:
            result_ = ["error","None"]
    else:
        result_ = ["pass",out_["지원센터코드"][0]]
    return result_

In [27]:
df_center = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\getSpctrInfo.xlsx")

In [28]:
target_col = ["지원센터코드","주소","지역코드","지원센터단위코드","지원센터구분코드"]
lambda_ = lambda x: False if x =="푸드뱅크" or x=="푸드마켓" else (True)
df_list_1 = df_center[df_center["지원센터코드"].map(lambda_)]
df_list_1 = df_list_1[target_col].groupby(by=target_col).count()#.unique()
df_list_1 = df_list_1.reset_index(level=[0]).reset_index()
df_list_1.head()

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드
0,전라남도 신안군 압해읍 구항길 92-58,전남,기초,푸드뱅크,1004섬신안기초푸드뱅크
1,경기도 가평군 가평읍 달전로 44 1층,경기,기초,푸드뱅크,가평군푸드뱅크
2,경기도 가평군 가평읍 달전로 44 1층(달전리),경기,기초,푸드뱅크,가평군푸드뱅크
3,서울특별시 강남구 개포로 605 강남 장애인 문화복지관 강남구기초푸드뱅크,서울,기초,푸드뱅크,강남구기초푸드뱅크
4,서울특별시 강남구 학동로 426 (삼성동) 강남구청내 강남사랑나눔이동푸드마켓,서울,기초,푸드마켓,강남이동푸드마켓


In [29]:
df_list_2 = df_center.copy()
list_ = df_list_2["주소"].map(lambda x: query(x))

In [30]:
df_list_2["지원센터코드2"] = [i[1] for i in list_]
df_list_2["estimate"] = [i[0] for i in list_]

In [31]:
df_list_2.head()

,Unnamed: 0,기준년월,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드,지원센터주소,지원센터상세주소,주소,지원센터주소2,지원센터주소3,지원센터코드2,estimate
0,0,201701,중앙,전국,푸드뱅크,전국푸드뱅크,서울특별시 마포구 만리재로 14,한국사회복지회관 4층,서울특별시 마포구 만리재로 14 한국사회복지회관 4층,서울특별시 마포구 만리재로 14,서울특별시 마포구 만리재로 14,전국푸드뱅크,pass
1,1,201701,서울,광역,푸드뱅크,서울시광역푸드뱅크,서울 도봉구 창동,1-7,서울 도봉구 창동 1-7,서울 도봉구 창동,서울 도봉구 창동,서울시광역푸드뱅크,pass
2,2,201701,서울,기초,푸드뱅크,영등포사랑나눔푸드뱅크,서울특별시 영등포구 도림로61길 7,7,서울특별시 영등포구 도림로61길 7 7,서울특별시 영등포구 도림로61길 7,서울특별시 영등포구 도림로 61길 7,영등포사랑나눔푸드뱅크,pass
3,3,201701,서울,기초,푸드뱅크,성동희망푸드뱅크,서울특별시 성동구 왕십리로28길 10 준희빌딩,1층,서울특별시 성동구 왕십리로28길 10 준희빌딩 1층,서울특별시 성동구 왕십리로28길 10,서울특별시 성동구 왕십리로 28길 10 준희빌딩,성동희망푸드뱅크,pass
4,4,201701,서울,기초,푸드뱅크,종로구푸드뱅크,서울 종로구 숭인1동,55-2 숭인1동주민센터 4층,서울 종로구 숭인1동 55-2 숭인1동주민센터 4층,서울 종로구 숭인1동,서울 종로구 숭인 1동,종로구 푸드뱅크마켓센터,multi


In [35]:
target_col2 = ["지역코드","지원센터단위코드","지원센터구분코드","지원센터코드2","주소"]
all_list = df_list_2[target_col2].groupby(by=target_col2).count()
all_list = all_list.reset_index(level=[0,1,2,3]).reset_index()
all_list

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드2
0,강원도 강릉시 구정면 구정중앙로 158-10,강원,광역,푸드뱅크,None
1,강원도 강릉시 난설헌로 12-6 (운정동)사랑관 1층,강원,광역,푸드뱅크,강원도광역푸드뱅크
2,강원 원주시 학성동 223-73 원주시사회복지센터 105호,강원,기초,푸드마켓,원주시푸드나눔센터
3,강원도 원주시 갈머리2길 31 (학성동),강원,기초,푸드마켓,원주시푸드나눔센터
4,강원 철원군 동송읍 이평로 80,강원,기초,푸드뱅크,None
...,...,...,...,...,...
755,충청북도 청주시 흥덕구 강서로71 20-3 (강서동),충북,기초,푸드뱅크,청주시사랑나눔기초푸드뱅크
756,충북 청주시 흥덕구 운천동 1486,충북,기초,푸드뱅크,청주시흥덕푸드뱅크
757,충청북도 청주시 상당구 상당로13 15 청주제일교회,충북,기초,푸드뱅크,청주제일기초푸드뱅크
758,충청북도 충주시 만리산15길 8 (교현동),충북,기초,푸드뱅크,충주시사회복지푸드뱅크


# Only considering "경기"

In [38]:
list_target = all_list[all_list["지역코드"]=="경기"].reset_index(drop=True)
list_target

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드2
0,경기도 화성시 봉담읍 덕우공단2길 48 덕우리 48-1,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고
1,경기도 화성시 정남면 괘랑1길 42-30 A동,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고
2,경기도 양주시 칠봉산로 303 (봉양동),경기,광역,푸드뱅크,경기광역기부식품등지원센터북부물류창고
3,"경기도 수원시 장안구 서부로 2139 6003,6004호 (율전동, 에스케이허브블루)",경기,광역,푸드뱅크,경기도광역푸드뱅크
4,경기도 수원시 장안구 서부로 2161 (89-5 6층),경기,광역,푸드뱅크,경기도광역푸드뱅크
...,...,...,...,...,...
185,경기도 화성시 향남읍 행정중앙1로 30,경기,기초,푸드뱅크,화성시행복나눔푸드뱅크
186,경기도 화성시 남여울3길 9-10 (능동),경기,기초,푸드뱅크,화성여럿이함께푸드뱅크
187,경기도 화성시 봉담읍 독정길 30 (왕림리),경기,기초,푸드뱅크,화성은혜푸드뱅크
188,경기도 화성시 봉담읍 독정길 30 화성은혜푸드뱅크,경기,기초,푸드뱅크,화성은혜푸드뱅크


# Crawling lat and lng

In [12]:
options = Options()
down_path = r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\GiveFood\Timeseries"
prefs = {"download.default_directory":down_path}
options.add_experimental_option("prefs",prefs)

In [13]:
driver = webdriver.Chrome(options=options)

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


In [69]:
url_base = "https://address.dawul.co.kr/"
driver.get(url_base)

In [71]:
test_ = list_target["주소"][1]

In [81]:
def Extract_code(test_):
    url_base = "https://address.dawul.co.kr/"
    driver.get(url_base)
    input_box = driver.find_elements(By.CSS_SELECTOR,"div input#input_juso")[0]
    input_box.send_keys(test_)
    time.sleep(2)
    click_box = driver.find_elements(By.CSS_SELECTOR,"div#btnSch")[0]
    click_box.click()
    time.sleep(3)
    adjusted_ = driver.find_elements(By.CSS_SELECTOR,"tr td#insert_data_3")[0].text
    lat_lng = driver.find_elements(By.CSS_SELECTOR,"tr td#insert_data_5")[0].text
    reg = re.compile("[XY] +:([0-9.]+)")
    try:
        X_,Y_ = reg.findall(lat_lng)
    except:
        X_,Y_ = "Error","Error"
    return adjusted_,X_,Y_

In [83]:
ll_list = []
tq = tqdm(list_target["주소"])
for i in tq:
    ad_,x_,y_ = Extract_code(i)
    print("Before: ",i,"\n  After: ",ad_)
    ll_list.append([ad_,x_,y_])
    print()

  0%|          | 0/190 [00:00<?, ?it/s]

Before:  경기도 화성시 봉담읍 덕우공단2길 48 덕우리 48-1 
  After:  경기도 화성시 봉담읍 덕우리 161-23 덕우리 48-1

Before:  경기도 화성시 정남면 괘랑1길 42-30 A동 
  After:  경기도 화성시 정남면 괘랑리 655-18 A동

Before:  경기도 양주시 칠봉산로 303 (봉양동) 
  After:  경기도 양주시 봉양동 558-1

Before:  경기도 수원시 장안구 서부로 2139 6003,6004호 (율전동, 에스케이허브블루) 
  After:  경기도 수원시 장안구 율전동 285-2 SK허브블루 6003,6004호

Before:  경기도 수원시 장안구 서부로 2161 (89-5 6층) 
  After:  경기도 수원시 장안구 율전동 89-5 (89-5 6층)

Before:  경기도 평택시 원평로28 34 (군문동) 
  After:  경기도 평택시 군문동 357-2

Before:  경기도 평택시 탄현1로 160 (신장동) B102호 
  After:  경기도 평택시 신장동 259-3 B102호

Before:  경기 고양시 일산동구 백석동 1414-9 
  After:  경기도 고양시 일산동구 호수로446번길 55 (백석동)

Before:  경기도 고양시 일산동구 호수로446 55 (백석동) 
  After:  경기도 고양시 일산동구 백석동 1414-9

Before:  경기 남양주시 진건읍 용정리 781-2 
  After:  경기도 남양주시 진건읍 사릉로 406-1 (용정리, 서부희망케어센터)

Before:  경기도 남양주시 진건읍 사릉로 406-1 (용정리) 
  After:  경기도 남양주시 진건읍 용정리 781-2

Before:  경기 남양주시 화도읍 마석우리 79 (마석 중앙로 79) 
  After:  전환결과없음

Before:  경기도 부천시 신흥로275 19 (중동) 
  After:  경기도 부천시 중동 1051-8

Before:  경기 시흥시 정왕동 1838-7 

Before:  경기도 성남시 분당구 미금로 246 (금곡동, 청솔마을주공6단지아파트) 
  After:  경기도 성남시 분당구 금곡동 126 청솔마을주공6단지아파트

Before:  경기도 이천시 마장면 서이천로 105-6 (이치리) 
  After:  경기도 이천시 마장면 이치리 411-6

Before:  경기도 이천시 마장면 작촌로 24-23  
  After:  경기도 이천시 마장면 오천리 270-46

Before:  경기도 포천시 소흘읍 송우로132 56 (송우리) 
  After:  경기도 포천시 소흘읍 송우리 589-5

Before:  경기도 포천시 소흘읍 송우로132 56 솔모루푸드뱅크 
  After:  경기도 포천시 소흘읍 송우리 589-5 솔모루푸드뱅크

Before:  경기 수원시 권선구 탑동 869-9 태영아원 102호 
  After:  경기도 수원시 권선구 효탑로 24-17 태영아원 102호 (탑동)

Before:  경기도 수원시 권선구 효탑로 24-17 태영아이원 102호(탑동) 
  After:  경기도 수원시 권선구 탑동 869-9 태영아이원 102호

Before:  경기도 수원시 장안구 장안로63 8 (영화동) 
  After:  경기도 수원시 장안구 영화동 394-5 8

Before:  경기도 수원시 팔달구 매산로 86 (매산로3가) 
  After:  경기도 수원시 팔달구 매산로3가 52-6

Before:  경기도 광명시 금하로527 12 (소하동) 
  After:  경기도 광명시 소하동 36-7

Before:  경기도 시흥시 목감초등길 45  
  After:  경기도 시흥시 조남동 산9-19

Before:  경기도 시흥시 목감초등길 45 (조남동, 목감종합사회복지관) 
  After:  경기도 시흥시 조남동 산9-19 목감종합사회복지관

Before:  경기 시흥시 정왕동 1608-5 101호 
  After:  경기도 시흥시 봉우재로 23-24 101호 (정왕동, 군서중5길 30)

Before:  

In [87]:
list_target["주소_after"] = [i[0] for i in ll_list]
list_target["lat"] = [i[1] for i in ll_list]
list_target["lng"] = [i[2] for i in ll_list]
list_target.head()

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드2,주소_after,lat,lng
0,경기도 화성시 봉담읍 덕우공단2길 48 덕우리 48-1,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고,경기도 화성시 봉담읍 덕우리 161-23 덕우리 48-1,126.917717,37.1558513
1,경기도 화성시 정남면 괘랑1길 42-30 A동,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고,경기도 화성시 정남면 괘랑리 655-18 A동,126.985065,37.1819922
2,경기도 양주시 칠봉산로 303 (봉양동),경기,광역,푸드뱅크,경기광역기부식품등지원센터북부물류창고,경기도 양주시 봉양동 558-1,127.071769,37.8597747
3,"경기도 수원시 장안구 서부로 2139 6003,6004호 (율전동, 에스케이허브블루)",경기,광역,푸드뱅크,경기도광역푸드뱅크,"경기도 수원시 장안구 율전동 285-2 SK허브블루 6003,6004호",126.971821,37.2994042
4,경기도 수원시 장안구 서부로 2161 (89-5 6층),경기,광역,푸드뱅크,경기도광역푸드뱅크,경기도 수원시 장안구 율전동 89-5 (89-5 6층),126.972395,37.3012605


In [88]:
list_target

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드2,주소_after,lat,lng
0,경기도 화성시 봉담읍 덕우공단2길 48 덕우리 48-1,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고,경기도 화성시 봉담읍 덕우리 161-23 덕우리 48-1,126.917717,37.1558513
1,경기도 화성시 정남면 괘랑1길 42-30 A동,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고,경기도 화성시 정남면 괘랑리 655-18 A동,126.985065,37.1819922
2,경기도 양주시 칠봉산로 303 (봉양동),경기,광역,푸드뱅크,경기광역기부식품등지원센터북부물류창고,경기도 양주시 봉양동 558-1,127.071769,37.8597747
3,"경기도 수원시 장안구 서부로 2139 6003,6004호 (율전동, 에스케이허브블루)",경기,광역,푸드뱅크,경기도광역푸드뱅크,"경기도 수원시 장안구 율전동 285-2 SK허브블루 6003,6004호",126.971821,37.2994042
4,경기도 수원시 장안구 서부로 2161 (89-5 6층),경기,광역,푸드뱅크,경기도광역푸드뱅크,경기도 수원시 장안구 율전동 89-5 (89-5 6층),126.972395,37.3012605
...,...,...,...,...,...,...,...,...
185,경기도 화성시 향남읍 행정중앙1로 30,경기,기초,푸드뱅크,화성시행복나눔푸드뱅크,경기도 화성시 향남읍 행정리 467,126.917942,37.1256661
186,경기도 화성시 남여울3길 9-10 (능동),경기,기초,푸드뱅크,화성여럿이함께푸드뱅크,경기도 화성시 능동 1158-3,127.054243,37.1985338
187,경기도 화성시 봉담읍 독정길 30 (왕림리),경기,기초,푸드뱅크,화성은혜푸드뱅크,경기도 화성시 봉담읍 왕림리 211,126.936551,37.1950498
188,경기도 화성시 봉담읍 독정길 30 화성은혜푸드뱅크,경기,기초,푸드뱅크,화성은혜푸드뱅크,경기도 화성시 봉담읍 왕림리 211 화성은혜푸드뱅크,126.936551,37.1950498


In [89]:
list_target.to_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\경기\lat_lng.xlsx")

In [111]:
list_target[list_target["lat"]=="Error"]

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드2,주소_after,lat,lng
92,경기도 동두천시 광암로6 2 (광암동),경기,기초,푸드뱅크,동두천천사푸드뱅크,,Error,Error
153,경기도 수원시 영통구 매봉로35 23-15 1층 (매탄동),경기,기초,푸드뱅크,영통푸드뱅크,,Error,Error
181,경기도 하남시 역말로 71 (덕풍동),경기,기초,푸드뱅크,하남시푸드뱅크,,Error,Error
182,경기도 하남시 역말로 71 (덕풍동)1층,경기,기초,푸드뱅크,하남시푸드뱅크,,Error,Error
189,경기도 성남시 수정구 희망로534 39 (산성동) 1층,경기,기초,푸드뱅크,희망등대푸드뱅크,,Error,Error


In [99]:
list_target["lat"][16] = 127.440114
list_target["lng"][16] = 37.2759339

In [100]:
list_target["lat"][46] = 127.068766
list_target["lng"][46] = 37.1513114

In [101]:
list_target["lat"][60] = 126.780100
list_target["lng"][60] = 37.6459972

In [103]:
list_target["lat"][88] = 127.345441
list_target["lng"][88] = 37.3486713

In [110]:
list_target["lat"][136] = 127.276145
list_target["lng"][136] = 37.0069142

list_target["lat"][164] = 127.066600
list_target["lng"][164] = 37.7457079

list_target["lat"][180] = 127.203655
list_target["lng"][180] = 37.5510495

In [114]:
list_target

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드2,주소_after,lat,lng
0,경기도 화성시 봉담읍 덕우공단2길 48 덕우리 48-1,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고,경기도 화성시 봉담읍 덕우리 161-23 덕우리 48-1,126.917717,37.1558513
1,경기도 화성시 정남면 괘랑1길 42-30 A동,경기,광역,푸드뱅크,경기광역기부식품등지원센터남부물류창고,경기도 화성시 정남면 괘랑리 655-18 A동,126.985065,37.1819922
2,경기도 양주시 칠봉산로 303 (봉양동),경기,광역,푸드뱅크,경기광역기부식품등지원센터북부물류창고,경기도 양주시 봉양동 558-1,127.071769,37.8597747
3,"경기도 수원시 장안구 서부로 2139 6003,6004호 (율전동, 에스케이허브블루)",경기,광역,푸드뱅크,경기도광역푸드뱅크,"경기도 수원시 장안구 율전동 285-2 SK허브블루 6003,6004호",126.971821,37.2994042
4,경기도 수원시 장안구 서부로 2161 (89-5 6층),경기,광역,푸드뱅크,경기도광역푸드뱅크,경기도 수원시 장안구 율전동 89-5 (89-5 6층),126.972395,37.3012605
...,...,...,...,...,...,...,...,...
185,경기도 화성시 향남읍 행정중앙1로 30,경기,기초,푸드뱅크,화성시행복나눔푸드뱅크,경기도 화성시 향남읍 행정리 467,126.917942,37.1256661
186,경기도 화성시 남여울3길 9-10 (능동),경기,기초,푸드뱅크,화성여럿이함께푸드뱅크,경기도 화성시 능동 1158-3,127.054243,37.1985338
187,경기도 화성시 봉담읍 독정길 30 (왕림리),경기,기초,푸드뱅크,화성은혜푸드뱅크,경기도 화성시 봉담읍 왕림리 211,126.936551,37.1950498
188,경기도 화성시 봉담읍 독정길 30 화성은혜푸드뱅크,경기,기초,푸드뱅크,화성은혜푸드뱅크,경기도 화성시 봉담읍 왕림리 211 화성은혜푸드뱅크,126.936551,37.1950498


In [113]:
list_target[list_target["lat"]=="Error"]

,주소,지역코드,지원센터단위코드,지원센터구분코드,지원센터코드2,주소_after,lat,lng
92,경기도 동두천시 광암로6 2 (광암동),경기,기초,푸드뱅크,동두천천사푸드뱅크,,Error,Error
153,경기도 수원시 영통구 매봉로35 23-15 1층 (매탄동),경기,기초,푸드뱅크,영통푸드뱅크,,Error,Error
181,경기도 하남시 역말로 71 (덕풍동),경기,기초,푸드뱅크,하남시푸드뱅크,,Error,Error
182,경기도 하남시 역말로 71 (덕풍동)1층,경기,기초,푸드뱅크,하남시푸드뱅크,,Error,Error
189,경기도 성남시 수정구 희망로534 39 (산성동) 1층,경기,기초,푸드뱅크,희망등대푸드뱅크,,Error,Error


In [118]:
lambda_2 = lambda x: str(x)
list_target["lat_lng"] = list_target["lat"].map(lambda_2) +"+" + list_target["lng"].map(lambda_2)

In [123]:
Counter(list_target["lat_lng"]).most_common()

[('Error+Error', 5),
 ('126.705841+37.7277236', 4),
 ('127.079393+36.9987004', 3),
 ('127.086751+36.9877833', 2),
 ('127.054000+37.0740890', 2),
 ('126.774253+37.5080269', 2),
 ('127.440114+37.2759339', 2),
 ('127.120640+37.4103988', 2),
 ('127.010425+37.2698987', 2),
 ('126.860097+37.4781574', 2),
 ('126.920488+37.3968530', 2),
 ('127.087729+37.7374054', 2),
 ('127.057026+37.7331894', 2),
 ('126.917942+37.1256661', 2),
 ('126.919580+37.1303978', 2),
 ('126.960005+37.2100087', 2),
 ('127.517915+37.8141473', 2),
 ('126.756267+37.6746371', 2),
 ('126.760809+37.6722138', 2),
 ('126.785244+37.6422406', 2),
 ('127.150095+37.5915565', 2),
 ('126.937801+37.3570655', 2),
 ('127.331255+37.6523476', 2),
 ('127.345441+37.3486713', 2),
 ('127.213172+37.2378580', 2),
 ('126.678850+37.1787971', 2),
 ('127.044366+37.1994659', 2),
 ('127.144998+36.9911783', 2),
 ('126.929869+36.9833502', 2),
 ('127.132523+37.8364603', 2),
 ('126.858531+37.3856668', 2),
 ('126.879888+37.3622642', 2),
 ('127.427990+37.0

In [124]:
len(Counter(list_target["lat_lng"]))

140

In [125]:
# 최소 145개의 주소가 있다 (이는 이사가거나 한 내용이 포함되어 있는 듯하다)

In [126]:
list_target.to_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\out\경기\lat_lng.xlsx")

# Visualization

In [128]:
col_list = ["red", "blue", "green", "purple", "orange", "darkred",
            "lightred", "beige", "darkblue", "darkgreen", "cadetblue",
            "darkpurple", "white", "pink", "lightblue", "lightgreen", "gray", "black", "lightgray"]

In [145]:
out = list_target.copy()
lng_list = []
lat_list = []
for row,col in out.iterrows():
    try:
        lat_ = float(col["lat"])
        lng_ = float(col["lng"])
    except:
        lat_ = 0
        lng_ = 0
    lat_list.append(lat_)
    lng_list.append(lng_)


In [155]:
### 위도 경도 위치 바뀌었음
out["lat"] = lng_list
out["lng"] = lat_list

In [156]:
lat_,lng_ = out.loc[0]["lat"],out.loc[0]["lng"]
g_map = folium.Map(location=[lat_,lng_],zoom_start=18,tiles="Stamen Toner")

In [157]:
marker = folium.Marker(location=[lat_, lng_],popup="A",icon= folium.Icon(color="green",icon="star"))
marker.add_to(g_map)

In [160]:
i = 0
tar_ = "경기"

tar_df = out[out["지역코드"]==tar_]
tar_df = tar_df[["지원센터단위코드","지원센터코드2","lat","lng"]]
tar_df["level_col"] = tar_df["지원센터단위코드"].map(lambda x: "blue" if x=="기초" else ("red"))
tar_df["shape"] = tar_df["지원센터단위코드"].map(lambda x: "home" if x=="기초" else ("flag"))
for row,col in tar_df.iterrows():
    tar_col = col["level_col"]
    tar_pop = col["지원센터코드2"]
    tar_shape = col["shape"]
    check_ = col["지원센터단위코드"]
    try:
        if check_ == "기초":
            marker = folium.Marker(location=[col["lat"], col["lng"]],popup=tar_pop,
                                   icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)
        else:
            marker = folium.Marker(location=[col["lat"], col["lng"]],popup=tar_pop,
                       icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)
            marker = folium.CircleMarker(location=[col["lat"], col["lng"]],popup=tar_pop
                                         ,radius=20
                                         ,icon= folium.Icon(color=tar_col,icon=tar_shape)).add_to(g_map)
    except:
        pass

In [161]:
g_map